# Introduction to LLMs Notebook

To use this notebook, you will need to install the following packages:

```bash
!pip install pandas
!pip install tqdm
!pip install altair
!pip install scikit-learn
|pip install ollama
```

## Install and Import Libraries

In [4]:
# Importing libraries
from rich.console import Console
import pandas as pd
from tqdm import tqdm
import altair as alt
import ollama
import ast
import pandas as pd
import time
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import warnings
warnings.filterwarnings('ignore')

console = Console()

## Load Datasets

You either have the option of using the premade dataset available in Google Drive (though you will need to change the path to the file) or running the code below to remake the dataset from scratch. 

**Be warned, this file is quite large because of the size of the novels, so you may want to use a subset of the novels to test this code.**

### Google Drive Dataset

You can download this dataset here [https://drive.google.com/file/d/1LkaRtYph_lWtMPRyzZpECuzEMD3WPx26/view?usp=sharing](https://drive.google.com/file/d/1LkaRtYph_lWtMPRyzZpECuzEMD3WPx26/view?usp=sharing) and it's very larger so make sure you don't push it up to GitHub.


In [2]:
combined_novels_nyt_df = pd.read_csv("combined_novels_nyt_with_text.csv")

In [3]:
combined_novels_nyt_genre_df = combined_novels_nyt_df[(combined_novels_nyt_df.cleaned_pg_eng_text.notna()) & (combined_novels_nyt_df.genre != "na")][['top_500_rank', 'title', 'author', 'pub_year', 'orig_lang', 'genre',
       'author_birth', 'author_death', 'author_gender', 'author_primary_lang',
       'author_nationality', 'author_field_of_activity', 'author_occupation',
       'oclc_holdings', 'oclc_eholdings', 'oclc_total_editions',
       'oclc_holdings_rank', 'oclc_editions_rank', 'gr_avg_rating',
       'gr_num_ratings', 'gr_num_reviews', 'gr_avg_rating_rank',
       'gr_num_ratings_rank', 'oclc_owi', 'author_viaf', 'gr_url', 'wiki_url',
       'pg_eng_url', 'pg_orig_url', 'year', 'week', 'rank', 'title_id',
       'nyt_title', 'pub_year_date', 'pub_date','pg_eng_text_len',
       'pg_orig_text_len', 'pg_eng_token_len', 'pg_orig_token_len',
       'cleaned_pg_eng_text', 'cleaned_pg_orig_text']]

combined_novels_nyt_genre_df['cleaned_pg_eng_text_len'] = combined_novels_nyt_genre_df['cleaned_pg_eng_text'].str.len()
combined_novels_nyt_genre_df['cleaned_orig_eng_text_len'] = combined_novels_nyt_genre_df['cleaned_pg_orig_text'].str.len()

In [5]:
def clean_text(text):
    """
    Cleans the input text by removing unnecessary metadata, special characters, and image references.

    Parameters
    ----------
    text : str
        The raw text string to clean.

    Returns
    -------
    str
        A cleaned version of the input text.
    """
    # Decode bytes to string if text is in bytes format
    if isinstance(text, bytes):
        text = text.decode('utf-8', errors='ignore')
    
    # Remove image references like "bookcover.jpg" or "p003.jpg (307K)"
    text = re.sub(r'\b\w+\.jpg\b\s*\(\d+K\)', '', text)
    text = re.sub(r'\b\w+\.jpg\b', '', text)

    # Remove sections like "Full Size" which appear frequently
    text = re.sub(r'\bFull Size\b', '', text, flags=re.IGNORECASE)
    
    # Remove phrases indicating editor notes or eBook-related content
    text = re.sub(r'(Ebook Editor’s Note|Project Gutenberg edition)', '', text, flags=re.IGNORECASE)
    
    # Remove special character sequences (e.g., \xe2\x80\x9c, etc.)
    text = re.sub(r'\\x[0-9A-Fa-f]{2}', '', text)
    text = re.sub(r'\\[a-zA-Z]', '', text)  # Remove any leftover backslashes
    
    # Remove extra newline characters and unnecessary whitespace
    text = re.sub(r'\n+', '\n', text).strip()  # Reduce multiple newlines to one
    text = re.sub(r'\s{2,}', ' ', text)  # Replace multiple spaces with a single space

    # Optional: Remove all caps headings if needed, which might signify metadata (like CHAPTER TITLES)
    text = re.sub(r'^[A-Z\s]{3,}$', '', text, flags=re.MULTILINE)

    return text

tqdm.pandas(desc="Cleaning Text")

combined_novels_nyt_genre_df.cleaned_pg_eng_text = combined_novels_nyt_genre_df.cleaned_pg_eng_text.fillna('')
combined_novels_nyt_genre_df.cleaned_pg_orig_text = combined_novels_nyt_genre_df.cleaned_pg_orig_text.fillna('')
combined_novels_nyt_genre_df['full_cleaned_pg_eng_text'] = combined_novels_nyt_genre_df.cleaned_pg_eng_text.progress_apply(clean_text)
combined_novels_nyt_genre_df['full_cleaned_pg_orig_text'] = combined_novels_nyt_genre_df.cleaned_pg_orig_text.progress_apply(clean_text)

Cleaning Text: 100%|██████████| 87/87 [00:02<00:00, 31.01it/s]


In [6]:
combined_novels_nyt_genre_df.genre.value_counts()

history       24
romance       12
action        10
bildung       10
fantasy        7
political      7
scifi          5
mystery        4
allegories     3
autobio        3
horror         2
Name: genre, dtype: int64

In [7]:
vectorizer = TfidfVectorizer(stop_words="english", min_df=1, max_df=0.7,)

# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(combined_novels_nyt_genre_df.full_cleaned_pg_eng_text.fillna(''))

# Convert the TF-IDF matrix to a DataFrame for better readability
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Add the titles to the DataFrame
tfidf_df['title'] = combined_novels_nyt_genre_df['title'].values

# Melt the DataFrame to get a long format DataFrame with terms and scores
melted_tfidf_df = tfidf_df.melt(id_vars=['title'], var_name='term', value_name='score')

# Sort the DataFrame by score in descending order
sorted_tfidf_no_stopwords_min_max_df = melted_tfidf_df.sort_values(by='score', ascending=False)

# Display the top 10 results
sorted_tfidf_no_stopwords_min_max_df.head(10)

,title,term,score
15942353,The Jungle,jurgis,0.949413
1660887,This Side of Paradise,amory,0.938376
1295317,"Through the Looking-Glass, and What Alice Foun...",alice,0.919699
1295257,Alice's Adventures in Wonderland,alice,0.911474
22207592,Of Human Bondage,philip,0.895389
10050016,White Fang,fang,0.878559
31973533,The Secret Agent,verloc,0.878165
6298634,The Pilgrim's Progress,chr,0.867819
15925817,The Red & the Black,julien,0.857932
1253008,Kidnapped: The Adventures of David Balfour,alan,0.834603


In [8]:

# Group the text data by genre and join the texts within each genre
grouped_texts = combined_novels_nyt_genre_df.groupby('genre')['full_cleaned_pg_eng_text'].apply(lambda texts: ' '.join(texts)).reset_index()

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english", min_df=1, max_df=0.7)

# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(grouped_texts['full_cleaned_pg_eng_text'].fillna(''))

# Convert the TF-IDF matrix to a DataFrame for better readability
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Add the genres to the DataFrame
tfidf_df['genre'] = grouped_texts['genre'].values

# Melt the DataFrame to get a long format DataFrame with terms and scores
melted_tfidf_df = tfidf_df.melt(id_vars=['genre'], var_name='term', value_name='score')

# Sort the DataFrame by score in descending order
sorted_tfidf_no_stopwords_min_max_df = melted_tfidf_df.sort_values(by='score', ascending=False)

# Display the top 10 results
sorted_tfidf_no_stopwords_min_max_df.head(10)

,genre,term,score
3403765,autobio,swann,0.688831
1469898,allegories,grandet,0.628138
160439,fantasy,alice,0.611279
2712973,scifi,pencroft,0.597450
780566,horror,christine,0.594813
3043260,action,sancho,0.552047
2481151,autobio,odette,0.505621
2879536,action,quixote,0.493032
1962606,political,jurgis,0.480073
4045793,fantasy,wendy,0.450921


In [13]:
# Group by genre and get the top 10 terms for each genre
top_terms_by_genre = sorted_tfidf_no_stopwords_min_max_df.groupby('genre').head(10)

# Print the results
for genre, group in top_terms_by_genre.groupby('genre'):
    console.print(f"In our dataset we have this many novels {len(combined_novels_nyt_genre_df[combined_novels_nyt_genre_df.genre == genre])} for genre: {genre}", style="bright_magenta")
    console.print(f"Top 10 terms for genre: {genre}", style="bright_magenta")
    console.print(group[['term', 'score']])
    console.print("\n")

In our dataset we have this many novels 10 for genre: action

Top 10 terms for genre: action

term     score
3043260        sancho  0.552047
2879536       quixote  0.493032
1978977           kim  0.309052
1312465          fogg  0.240187
3105014          seor  0.173718
2695847  passepartout  0.156205
1236796          fang  0.150566
1748571          huck  0.117994
2007819          lama  0.117008
1949068           jim  0.105786

In our dataset we have this many novels 3 for genre: allegories

Top 10 terms for genre: allegories

...

Top 10 terms for genre: romance

term     score
2058879         levin  0.361453
606681         boffin  0.316385
3973979       vronsky  0.211128
549745          bella  0.186235
1237333         fanny  0.179559
4039264          wegg  0.177372
1120634        elinor  0.175837
1651692         hetty  0.168472
385317   arkadyevitch  0.131648
490675      bathsheba  0.131034

In our dataset we have this many novels 5 for genre: scifi

Top 10 terms for genre: scifi

term     score
2712973    pencroft  0.597450
1546368     harding  0.320924
931028        cyrus  0.293458
3291100     spilett  0.255352
50555    _nautilus_  0.248632
1616900     herbert  0.227483
2381939         neb  0.222443
2389342        nemo  0.190591
467499       ayrton  0.172882
853995      conseil  0.133675

In [11]:
# Filter the dataset for action and romance genres
filtered_df = combined_novels_nyt_genre_df[combined_novels_nyt_genre_df['genre'].isin(['action', 'romance'])]

# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words="english", min_df=1, max_df=0.7)
X = vectorizer.fit_transform(filtered_df['full_cleaned_pg_eng_text'].fillna(''))

# Encode the genres as binary labels
y = filtered_df['genre'].apply(lambda x: 1 if x == 'action' else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred, target_names=['romance', 'action']))

# Extract the most distinctive coefficients
feature_names = vectorizer.get_feature_names_out()
coefficients = log_reg.coef_.flatten()
coeff_df = pd.DataFrame({'term': feature_names, 'coefficient': coefficients})

# Sort by absolute value of coefficients to get the most distinctive terms
coeff_df['abs_coefficient'] = coeff_df['coefficient'].abs()
sorted_coeff_df = coeff_df.sort_values(by='abs_coefficient', ascending=False)

# Display the top 10 most distinctive terms
console.print("Top 10 most distinctive terms for action vs romance:")
console.print(sorted_coeff_df.head(10))

# Display the top 10 terms for action
console.print("\nTop 10 terms for action:")
console.print(sorted_coeff_df[sorted_coeff_df['coefficient'] > 0].head(10))

# Display the top 10 terms for romance
console.print("\nTop 10 terms for romance:")
console.print(sorted_coeff_df[sorted_coeff_df['coefficient'] < 0].head(10))

              precision    recall  f1-score   support

     romance       0.75      1.00      0.86         3
      action       1.00      0.50      0.67         2

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5



Top 10 most distinctive terms for action vs romance:

term  coefficient  abs_coefficient
81961         mrs    -0.458468         0.458468
43752        fang     0.348776         0.348776
70298         kim     0.330332         0.330332
46368        fogg     0.315881         0.315881
132712        tom     0.304972         0.304972
23468        buck     0.302627         0.302627
73037       levin    -0.231547         0.231547
43799       fanny    -0.225164         0.225164
17571   bathsheba    -0.224674         0.224674
67359      island     0.223476         0.223476

Top 10 terms for action:

term  coefficient  abs_coefficient
43752           fang     0.348776         0.348776
70298            kim     0.330332         0.330332
46368           fogg     0.315881         0.315881
132712           tom     0.304972         0.304972
23468           buck     0.302627         0.302627
67359         island     0.223476         0.223476
94876   passepartout     0.205433         0.205433
61866           huck     0.182405         0.182405
130239          thou     0.155512         0.155512
71385           lama     0.146316         0.146316

Top 10 terms for romance:

term  coefficient  abs_coefficient
81961         mrs    -0.458468         0.458468
73037       levin    -0.231547         0.231547
43799       fanny    -0.225164         0.225164
17571   bathsheba    -0.224674         0.224674
3631         adam    -0.218932         0.218932
105066  rochester    -0.218286         0.218286
21858      boffin    -0.203912         0.203912
39767   elizabeth    -0.203224         0.203224
11560        anne    -0.199121         0.199121
69997        kate    -0.186628         0.186628

In [12]:


# Assuming combined_novels_nyt_genre_df is already defined and contains 'full_cleaned_pg_eng_text' and 'genre' columns

# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words="english", min_df=1, max_df=0.7)
X = vectorizer.fit_transform(combined_novels_nyt_genre_df['full_cleaned_pg_eng_text'].fillna(''))

# Encode the genres as labels
y = combined_novels_nyt_genre_df['genre']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform Random Forest classification
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_clf.predict(X_test)

# Print classification report
console.print(classification_report(y_test, y_pred))

# Extract feature importances
feature_names = vectorizer.get_feature_names_out()
importances = rf_clf.feature_importances_
importance_df = pd.DataFrame({'term': feature_names, 'importance': importances})

# Sort by importance to get the most distinctive terms
sorted_importance_df = importance_df.sort_values(by='importance', ascending=False)

# Display the top 10 most distinctive terms
console.print("Top 10 most distinctive terms:")
console.print(sorted_importance_df.head(10))

precision    recall  f1-score   support

      action       0.00      0.00      0.00         4
  allegories       0.00      0.00      0.00         1
     autobio       1.00      1.00      1.00         1
     bildung       0.00      0.00      0.00         0
     fantasy       0.00      0.00      0.00         0
     history       0.42      1.00      0.59         5
      horror       0.00      0.00      0.00         1
     mystery       0.00      0.00      0.00         1
   political       0.00      0.00      0.00         2
     romance       1.00      0.33      0.50         3

    accuracy                           0.39        18
   macro avg       0.24      0.23      0.21        18
weighted avg       0.34      0.39      0.30        18

Top 10 most distinctive terms:

term  importance
78132        comrade    0.002868
349101        timeto    0.002363
315531         sugar    0.002211
270276      rattling    0.002150
161978          host    0.002132
249094         pages    0.002034
162061         hotel    0.001930
257295         plane    0.001892
348181       thyself    0.001822
334056  themountains    0.001754

In [15]:
marc_records_df = pd.read_csv("merged_preidentified_periodicals.csv")
console.print(f"Whe have this many marc records {len(marc_records_df)}")

Whe have this many marc records 66

In [19]:
def process_marc_record(row: pd.Series) -> pd.Series:
	"""
	Function to process a MARC record row to generate a human-readable description and classify the periodical.

	Parameters
	----------
	row : pd.Series
		A row of a dataframe with MARC record fields.

	Returns
	-------
	pd.Series
		A series with the human-readable description and classification from the Llama model.
	"""
	time.sleep(1)  # Avoid hitting API rate limits
	try:
		console.print(f"Processing periodical {row['periodical_name']}")
		
		knowledege_check_content = (
			   f"Based on this periodical name: {row['periodical_name']}, what do you know about this publication? "
		)
		console.print(f"Knowledge check content: {knowledege_check_content}", style="bold green")
		
		# Prepare the Ollama API request for the knowledge check
		knowledge_check_response = ollama.chat(
			model='llama3.2',
			messages=[
				{
					'role': 'system',
					'content': "You are a librarian with knowledge of periodicals. Return your answer strictly in JSON format like this: {'knowledge': 'Your knowledge here'}. Do not include any other text or explanations outside the JSON format."
				},
				{
					'role': 'user',
					'content': knowledege_check_content
				}
			]
		)
		console.print(f"Knowledge check response: {knowledge_check_response}", style="bold green")
		# Attempt to parse response as JSON
		try:
			knowledge_message = knowledge_check_response['message']['content']
			row['knowledge'] = ast.literal_eval(knowledge_message).get('knowledge', 'No knowledge available')
			console.print(f"Knowledge: {row.knowledge}", style="bold green")
		except (ValueError, SyntaxError):
			console.print("Error parsing JSON response for knowledge check.", style="bold red")
			row['knowledge'] = "No valid JSON returned."
		# Compile MARC record fields into a structured text for prompting
		marc_fields = '\n'.join([f"{field}: {value}" for field, value in row.items() if pd.notna(value) and field != "publication_type"])
		

		# Define the prompt content for generating a human-readable description
		description_content = (
			f"Here is combined data from multiple MARC records and HathiTrust data:\n{marc_fields}\nPlease create a brief human-readable description "
			f"including title, author, publication date, and subjects."
		)
		console.print(f"Description content: {description_content}", style="bold green")

		# Prepare the Ollama API request for the description
		description_response = ollama.chat(
			model='llama3.2',
			messages=[
				{
					'role': 'system',
					'content': "You are a librarian skilled in converting metadata from MARC files into user-friendly summaries. Please return your answer in strict JSON format like this: {'description': 'Your description here'}. Do not include any other text or explanations outside the JSON format."
				},
				{
					'role': 'user',
					'content': description_content
				}
			]
		)
		console.print(f"Description response: {description_response}", style="bold green")
		# Attempt to parse response as JSON
		description_message = description_response['message']['content']
		
		try:
			# Extract and store the human-readable description
			row['human_readable_description'] = ast.literal_eval(description_message).get('description', 'No description available')
			console.print(f"Description: {row.human_readable_description}", style="bold green")
		except (ValueError, SyntaxError):
			console.print("Error parsing JSON response for description.", style="bold red")
			row['human_readable_description'] = "No valid JSON returned."

		# Define the prompt content for genre classification
		classification_content = (
			f"Based on this description, classify the periodical into a genre: Information Bulletin, Radical Periodical, News & Politics Magazine.\n"
			f"Description: {row['human_readable_description']}"
		)

		# Prepare the Ollama API request for classification
		classification_response = ollama.chat(
			model='llama3.2',
			messages=[
				{
					'role': 'system',
					'content': "You are a classification expert who assigns genres to periodicals. "
							   "Return your answer strictly in JSON format like this: {'classification': 'Your classification here'}."
							   "Do not include any other text or explanations outside the JSON format."
				},
				{
					'role': 'user',
					'content': classification_content
				}
			]
		)

		# Attempt to parse response as JSON
		classification_message = classification_response['message']['content']
		
		try:
			row['classification'] = ast.literal_eval(classification_message).get('classification', 'Unclassified')
			console.print(f"Classification: {row.classification}", style="bold green")
			console.print(f"Actual classification: {row['publication_type']}", style="bright_magenta")
			console.print("\n")
		except (ValueError, SyntaxError):
			console.print("Error parsing JSON response for classification.", style="bold red")
			row['classification'] = "No valid JSON returned."

	except Exception as e:
		console.print(f"Unexpected error: {e}", style="bold red")
		row['human_readable_description'] = None
		row['classification'] = None

	return row


# Process each MARC record
marc_df = marc_records_df[0:10].apply(process_marc_record, axis=1)

Processing periodical AIM information bulletin

Knowledge check content: Based on this periodical name: AIM information bulletin, what do you know about this 
publication? 

Knowledge check response: {'model': 'llama3.2', 'created_at': '2024-11-14T18:36:26.735061Z', 'message': {'role': 
'assistant', 'content': "{'knowledge': {'title': 'AIM Information Bulletin', 'publisher': 'American Immunological 
Association (AIA)', 'frequency': 'bimonthly', 'circulation': '1,500-3,000', 'history': 'Published from 1948 to 1976
and revived in 1987.', 'content': 'Original research articles, review articles, case reports, and proceedings of 
AIA conferences on immunology.'}}"}, 'done_reason': 'stop', 'done': True, 'total_duration': 9893991375, 
'load_duration': 31743208, 'prompt_eval_count': 87, 'prompt_eval_duration': 1915974000, 'eval_count': 91, 
'eval_duration': 7942370000}

Knowledge: {'title': 'AIM Information Bulletin', 'publisher': 'American Immunological Association (AIA)', 
'frequency': 'bimonthly', 'circulation': '1,500-3,000', 'history': 'Published from 1948 to 1976 and revived in 
1987.', 'content': 'Original research articles, review articles, case reports, and proceedings of AIA conferences 
on immunology.'}

Description content: Here is combined data from multiple MARC records and HathiTrust data:
periodical_name: AIM information bulletin
001: 2698052.0
updated_ht_bib_key: 002698052
link: https://hdl.handle.net/2027/uc1.c088753409, https://hdl.handle.net/2027/uc1.31158013169338, 
https://hdl.handle.net/2027/inu.30000117848279, https://hdl.handle.net/2027/uc1.l0068269067, 
https://hdl.handle.net/2027/uc1.l0068269125, https://hdl.handle.net/2027/uc1.l0068269026, 
https://hdl.handle.net/2027/inu.30000117848261, https://hdl.handle.net/2027/uc1.c006804189, 
https://hdl.handle.net/2027/inu.30000117848329, https://hdl.handle.net/2027/uc1.l0060235397, 
https://hdl.handle.net/2027/inu.30000117848337, https://hdl.handle.net/2027/inu.30000100306202, 
https://hdl.handle.net/2027/inu.30000117848295, https://hdl.handle.net/2027/inu.30000100306194, 
https://hdl.handle.net/2027/uc1.31158012522107, https://hdl.handle.net/2027/inu.30000117848253, 
https://hdl.handle.net/2027/inu.30000117848287
htid: inu.30000100306194, inu.30000117848279, inu.30000117848295, inu.30000117848287, uc1.l0068269067, 
uc1.c006804189, uc1.l0068269026, inu.30000117848261, inu.30000117848329, uc1.c088753409, inu.30000100306202, 
inu.30000117848253, uc1.31158013169338, uc1.l0060235397, uc1.31158012522107, inu.30000117848337, uc1.l0068269125
date: no.091-122 yr.1984-86, Suppl. no.78, no.102-113 1985, no.079 yr.1983 mi.supp, no.077-079 yr.1982, no.24-25 
1978, no.79-90 1983, no.12-30 (July 1977-Dec.1978) NO.12-30 (JULY 1977-DEC.1978), no.126-137 1987, no.136 (Nov 
1987), no.45-54 1980, no.91-101 1984, no.075 yr.1982, no.114-125 1986, no.084 yr.1983, no.69 yr.1982 mi.supp., 
no.16-81 inc. 1977-83
original_source: 
                       University of California
                      , 
                       Indiana University
                      
record_url: https://catalog.hathitrust.org/Record/002698052
id: inu.30000100306194, inu.30000117848279, inu.30000117848295, inu.30000117848287, uc1.l0068269067, 
uc1.c006804189, uc1.l0068269026, inu.30000117848261, inu.30000117848329, uc1.c088753409, inu.30000100306202, 
inu.30000117848253, uc1.31158013169338, uc1.l0060235397, uc1.31158012522107, inu.30000117848337, uc1.l0068269125
metadata_schema_version: https://schemas.hathitrust.org/EF_Schema_MetadataSubSchema_v_3.0
enumeration_chronology: no.091-122 yr.1984-86, Suppl. no.78, no.102-113 1985, no.079 yr.1983 mi.supp, no.077-079 
yr.1982, no.24-25 1978, no.79-90 1983, no.12-30 (July 1977-Dec.1978) NO.12-30 (JULY 1977-DEC.1978), no.126-137 
1987, no.136 (Nov 1987), no.45-54 1980, no.91-101 1984, no.075 yr.1982, no.114-125 1986, no.084 yr.1983, no.69 
yr.1982 mi.supp., no.16-81 inc. 1977-83
type_of_resource: http://id.loc.gov/ontologies/bibframe/Text
title: AIM information bulletin /
date_created: 20200209.0
pub_date: 1984.0, 1985.0, 1986.0, 1987.0, 1988.0, 1978.0, 1980.0, 1982.0, 1983.0
language: eng
access_profile: google
lccn: 85647346
oclc: 12339773.0
page_count: 32.0, 354.0, 34.0, 322.0, 72.0, 232.0, 426.0, 332.0, 300.0, 46.0, 528.0, 242.0, 52.0, 20.0, 22.0, 158.0
feature_schema_version: https://schemas.hathitrust.org/EF_Schema_FeaturesSubSchema_v_3.0
access_rights: und, ic
alternate_title: ['Mozambique news', 'Agência de Informação de Moçambique information bulletin.', 'A.I.M. 
information bulletin.', 'AIM bulletin.']
category: Africa
genre_ld: ['http://id.loc.gov/vocabulary/marcgt/doc', 'http://id.loc.gov/vocabulary/marcgt/gov']
genre: ['document (computer)', 'government publication']
contributor_ld: {'id': 'http://www.viaf.org/viaf/148452547', 'type': 
'http://id.loc.gov/ontologies/bibframe/Organization', 'name': 'Agência de Informação de Moçambique.'}
contributor: Agência de Informação de Moçambique.
handle_url: http://hdl.handle.net/2027/inu.30000117848295, http://hdl.handle.net/2027/inu.30000117848337, 
http://hdl.handle.net/2027/inu.30000117848261, http://hdl.handle.net/2027/inu.30000117848287, 
http://hdl.handle.net/2027/uc1.l0068269026, http://hdl.handle.net/2027/uc1.3

...

Description content: Here is combined data from multiple MARC records and HathiTrust data:
periodical_name: Azania news
001: 2707548.0
updated_ht_bib_key: 002707548
link: https://babel.hathitrust.org/cgi/pt?id=inu.30000004998252, 
https://babel.hathitrust.org/cgi/pt?id=uiug.30112107803469, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000046781955, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000028478695, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000028509796, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000028509788, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000028478687, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000004998245, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000028478703, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000028478711, 
https://babel.hathitrust.org/cgi/pt?id=inu.30000028509804
htid: inu.30000028509796, uiug.30112107803469, inu.30000004998245, inu.30000046781955, inu.30000028478711, 
inu.30000028509804, inu.30000028478695, inu.30000028478703, inu.30000028478687, inu.30000028509788, 
inu.30000004998252
date: v.18,no.8 1982, v.8,no.7 1973, v.2,no.9 1967, v.3,no.15 1968, v.3,no.3-9 1968, v.18,no.11 1982, v.11:4-V.21:1
(INCOMPL.), v.26,no.9 1991, v.3,no.11-12 1968, v.9-13 inc. 1974-1978, v.8,no.1-5 1973
original_source: Indiana University, University of Illinois at Urbana-Champaign
record_url: https://catalog.hathitrust.org/Record/002707548
id: inu.30000028478687
metadata_schema_version: https://schemas.hathitrust.org/EF_Schema_MetadataSubSchema_v_3.0
enumeration_chronology: v.2,no.9 1967
type_of_resource: http://id.loc.gov/ontologies/bibframe/Text
title: Azania news.
date_created: 20200209.0
pub_date: 1967.0
language: eng
access_profile: google
issn: 0572-2853
lccn: sn 91028162
oclc: 2038783.0
page_count: 80.0
feature_schema_version: https://schemas.hathitrust.org/EF_Schema_FeaturesSubSchema_v_3.0
access_rights: ic
category: Africa
genre_ld: http://id.loc.gov/vocabulary/marcgt/doc
genre: ['document (computer)']
contributor_ld: [{'id': 'http://www.viaf.org/viaf/136322235', 'type': 
'http://id.loc.gov/ontologies/bibframe/Organization', 'name': 'Pan Africanist Congress of Azania. Revolutionary 
Command.'}, {'id': 'http://www.viaf.org/viaf/131418472', 'type': 
'http://id.loc.gov/ontologies/bibframe/Organization', 'name': 'Pan Africanist Congress of Azania.'}]
contributor: ['Pan Africanist Congress of Azania. Revolutionary Command.', 'Pan Africanist Congress of Azania.']
handle_url: http://hdl.handle.net/2027/inu.30000028478687
source_institution_ld: {'type': 'http://id.loc.gov/ontologies/bibframe/Organization', 'name': 'INU'}
source_institution: INU
lcc: DT1177.A92
type: ['DataFeedItem', 'PublicationVolume']
is_part_of: {'id': 'http://www.worldcat.org/oclc/2038783', 'type': 'CreativeWorkSeries', 'journalTitle': 'Azania 
news (Pan Africanist Congress of Azania)'}
last_rights_update_date: 20180823.0
pub_place_ld: {'id': 'http://id.loc.gov/vocabulary/countries/za', 'type': 
'http://id.loc.gov/ontologies/bibframe/Place', 'name': 'Zambia'}
pub_place: Zambia
main_entity_of_page: ['https://catalog.hathitrust.org/Record/002707548', 
'http://catalog.hathitrust.org/api/volumes/brief/oclc/2038783.json', 
'http://catalog.hathitrust.org/api/volumes/full/oclc/2038783.json']
publisher_ld: {'id': 
'http://catalogdata.library.illinois.edu/lod/entities/ProvisionActivityAgent/ht/Pan%20Africanist%20Congress%20of%20
Azania%20(S.A.)', 'type': 'http://id.loc.gov/ontologies/bibframe/Organization', 'name': 'Pan Africanist Congress of
Azania (S.A.)'}
publisher: Pan Africanist Congress of Azania (S.A.)
lowercase_periodical_name: azania_news
publication_directory: datasets/ht_ef_datasets/azania_news
volume_directory: inu_30000028478695, inu_30000028478711, inu_30000046781955, inu_30000004998245, 
uiug_30112107803469, inu_30000028478703, inu_30000028509788, inu_30000004998252, inu_30000028509796, 
inu_30000028509804, inu_30000028478687
ht_bib_key: 2707548.0
publishDates: 1966.0
974_y: 1967.0
lastUpdate: 20240328.0, 202303

Description response: {'model': 'llama3.2', 'created_at': '2024-11-14T18:41:17.363398Z', 'message': {'role': 
'assistant', 'content': "{'description': 'Azania news is a South African weekly newspaper focusing on current 
events, politics, business, and social issues affecting the African continent.'}"}, 'done_reason': 'stop', 'done': 
True, 'total_duration': 22538215375, 'load_duration': 42897583, 'prompt_eval_count': 1827, 'prompt_eval_duration': 
18423767000, 'eval_count': 32, 'eval_duration': 4064491000}

Description: Azania news is a South African weekly newspaper focusing on current events, politics, business, and 
social issues affecting the African continent.

Classification: News & Politics Magazine

Actual classification: Information Bulletin